# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [20]:
import pandas as pd

#url1 
url= 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv' 
df = pd.read_csv(url)

df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [21]:
df_filtered = df[
    (df['Total Claim Amount'] < 1000) &
    (df['Response'] == 'Yes')
]

print(df_filtered.head())
print(df_filtered.shape)  

    Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    Coverage Education Effective To Date EmploymentStatus Gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11         Employed      F  ...   

    Number of Open Complaints Number of Policies     Policy Type  \
3                         0.0                  2  Corp

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [26]:
#grouped = df.groupby('monthly_premium','policy_type','gender')
#df.groupby('monthly_premium','policy_type','gender)[numeric_columns].min()
#df.groupby('Monthly Premium Auto').agg({"Policy Type": 'mean', "Gender": 'mean'}).roun(2)
summary_avg = df.groupby(['Policy Type', 'Gender']).agg({'Monthly Premium Auto': 'mean',
    'Customer Lifetime Value': 'mean'
}).round(2)

print(summary_avg)

print(df)

                       Monthly Premium Auto  Customer Lifetime Value
Policy Type    Gender                                               
Corporate Auto F                      91.38                  7980.31
               M                      94.76                  7750.74
Personal Auto  F                      93.15                  8074.66
               M                      93.30                  7971.39
Special Auto   F                      93.56                  8460.40
               M                      93.20                  9010.60
       Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
0               0  DK49336     Arizona              4809.216960       No   
1               1  KX64629  California              2228.525238       No   
2               2  LZ68649  Washington             14947.917300       No   
3               3  XL78013      Oregon             22332.439460      Yes   
4               4  QA50777      Oregon              9025.067525     

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [27]:
state_counts = df['State'].value_counts()
print(state_counts)

California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: State, dtype: int64


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [31]:
clv_summary = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).round(2)
print(clv_summary)

                                  max      min   median
Education            Gender                            
Bachelor             F       73225.96  1904.00  5640.51
                     M       67907.27  1898.01  5548.03
College              F       61850.19  1898.68  5623.61
                     M       61134.68  1918.12  6005.85
Doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5577.67
High School or Below F       55277.45  2144.92  6039.55
                     M       83325.38  1940.98  6286.73
Master               F       51016.07  2417.78  5729.86
                     M       50568.26  2272.31  5579.10


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here